## Configure the sql server credentials

In [ ]:
sql_server <- '<input the ip of the sql server on your paper clip>'
user_name <- '<input the user name of the sql server on your paper clip>'
password <- '<input the password of the sql server user on your paper clip>'

## Install necessary libraries if missing

In [ ]:
# PreRequisites: You have installed Revolution R Enterprise 7.5.0 or higher on the machine and SQL Server 2016 CTP3 or higher on the database server
# Install required R libraries for this walkthrough if they are not installed. 

if (!('ROCR' %in% rownames(installed.packages()))){
  install.packages('ROCR', lib='./libsforjupyter')
}
if (!('RODBC' %in% rownames(installed.packages()))){
  install.packages('RODBC', lib='./libsforjupyter')
}



In [ ]:
# Add local lib directory to libPaths. The Jupyter windows user (LOCAL SERVICE) does not have write permission to global lib
.libPaths( c( .libPaths(), "./libsforjupyter") )

## Confirm that you are running under your own directory

In [ ]:
# Confirm you are working in your directory named after your username
# The username is also used as prefix for DB objects you will be creating in the tutorial 
username <- gsub("[^a-zA-Z0-9]", "", basename(getwd()))
username

## Specifying the Database Connection

* RODBC style connection string
* Compute context can be
    * Server: rx Commands will run on the server closer to the data
    * Local: Commands will run locally on the client. Data need to brought to the client. 




In [ ]:
connStr <- paste("Driver=SQL Server;Server=", sql_server, ";Database=nyctaxi;Uid=", user_name, ";Pwd=", password, sep="")

# Set ComputeContext. Needs a temp directory path to serialize R objects back and forth
sqlShareDir <- paste("C:\\AllShare\\",Sys.getenv("USERNAME"),sep="")
sqlWait <- TRUE
sqlConsoleOutput <- FALSE
cc <- RxInSqlServer(connectionString = connStr, shareDir = sqlShareDir, 
                    wait = sqlWait, consoleOutput = sqlConsoleOutput)
rxSetComputeContext(cc)

## Setting up R environment to query the database

* Define a DataSource (from a select query) to be used to explore the data and generate features from.
* Keep in mind that inDataSource is just a reference to the result dataset from the SQL query.

In [ ]:

sampleDataQuery <- "select tipped, tip_amount, fare_amount, passenger_count,trip_time_in_secs,trip_distance, 
pickup_datetime, dropoff_datetime, 
cast(pickup_longitude as float) as pickup_longitude, 
cast(pickup_latitude as float) as pickup_latitude, 
cast(dropoff_longitude as float) as dropoff_longitude, 
cast(dropoff_latitude as float)  as dropoff_latitude,
cast(pickup_longitude as float) * 100 as pickup_longitude2, 
cast(pickup_latitude as float) * 100 as pickup_latitude2, 
cast(dropoff_longitude as float) * 100 as dropoff_longitude2, 
cast(dropoff_latitude as float) * 100 as dropoff_latitude2,
payment_type from nyctaxi_sample
tablesample (5 percent) repeatable (98052)
"

ptypeColInfo <- list(
  payment_type = list(
    type = "factor",
    levels = c("CSH", "CRD", "DIS", "NOC", "UNK"),
    newLevels= c("CSH", "CRD", "DIS", "NOC", "UNK")
  )
)

inDataSource <- RxSqlServerData(sqlQuery = sampleDataQuery, connectionString = connStr, colInfo = ptypeColInfo,
                                colClasses = c(pickup_longitude = "numeric", pickup_latitude = "numeric", 
                                               dropoff_longitude = "numeric", dropoff_latitude = "numeric"),
                                rowsPerRead=500)


## Understand the data

These functions give you basic information about your data source, such as its size and the names of columns in the set:

- `rxGetInfo`
- `rxGetVarInfo` (same as `rxGetInfo` with `getVarInfo = TRUE`)
- `rxGetVarNames`

The `rxGetInfo` function offers information on the data location, size, number of columns, and other related information.
    
## Numerical Data Summaries

We now move to quantitative summaries of the data. The `rxSummary` command provides output on the number of observations contained in a data set and for a particular column, the function provides output on the following:

 - Name
 - Mean value
 - Standard Deviation
 - Minimum and Maximum value
 - Number of valid observations
 - Number of missing observations

The `rxSummary` function takes a formula as its first argument, and the name of the data set as the second.

In addition, column subsets and transformations may also be computed as a sub-call to the function using the transforms (and so forth) commands.

If compute context is SQL server, it is executed closer to the data. 


In [ ]:
################################
#        Data exploration      #
################################
# Summarize the Sampled Data
rxGetVarInfo(data = inDataSource)
rxSummary(~fare_amount:F(passenger_count,1,6), data = inDataSource)

In [ ]:
# Understand the relationship between tip amount, passenger count, and trip distance in the sample data
cube1 <- rxCube(tip_amount~F(passenger_count,1,6):F(trip_distance,0, 25), data = inDataSource)
cubePlot <- rxResultsDF(cube1)
cubePlot

In [ ]:
################################
#       Data Visualization     #
################################
# Plot fare amount histogram on the SQL Server, and ship the plot to R client to display
rxHistogram(~tip_amount | payment_type, data = inDataSource, title = "Tip Amount Histogram")
rxHistogram(~fare_amount, data = inDataSource, title = "Fare Amount Histogram")

## Feature engineering

### Feature engineering using functions defined in open source R

In [ ]:
# Define a function in open source R to calculate the direct distance between pickup and dropoff as a new feature 
# Use Haversine Formula: https://en.wikipedia.org/wiki/Haversine_formula
env <- new.env()

env$ComputeDist <- function(pickup_long, pickup_lat, dropoff_long, dropoff_lat){
  R <- 6371/1.609344 #radius in mile
  delta_lat <- dropoff_lat - pickup_lat
  delta_long <- dropoff_long - pickup_long
  degrees_to_radians = pi/180.0
  a1 <- sin(delta_lat/2*degrees_to_radians)
  a2 <- as.numeric(a1)^2
  a3 <- cos(pickup_lat*degrees_to_radians)
  a4 <- cos(dropoff_lat*degrees_to_radians)
  a5 <- sin(delta_long/2*degrees_to_radians)
  a6 <- as.numeric(a5)^2
  a <- a2+a3*a4*a6
  c <- 2*atan2(sqrt(a),sqrt(1-a))
  d <- R*c
  return (d)
}

featuretable <- paste0(username , "_features")
#Define the featureDataSource to be used to store the features, specify types of some variables as numeric
featureDataSource <- RxSqlServerData(table = featuretable, 
                                    colClasses = c(pickup_longitude = "numeric", pickup_latitude = "numeric", 
                                                   dropoff_longitude = "numeric", dropoff_latitude = "numeric",
                                                   passenger_count  = "numeric", trip_distance  = "numeric",
                                                   trip_time_in_secs  = "numeric", direct_distance  = "numeric"),
                                    connectionString = connStr)

# Create feature (direct distance) by calling rxDataStep() function, which calls the env$ComputeDist function to process records
# And output it along with other variables as features to the featureDataSource
# This will be the feature set for training machine learning models
start.time <- proc.time()
rxDataStep(inData = inDataSource, outFile = featureDataSource,  overwrite = TRUE, 
           varsToKeep=c("tipped", "fare_amount", "passenger_count","trip_time_in_secs", 
                        "trip_distance", "pickup_datetime", "dropoff_datetime", "pickup_longitude",
                        "pickup_latitude","dropoff_longitude", "dropoff_latitude"),
           transforms = list(direct_distance=ComputeDist(pickup_longitude, pickup_latitude, dropoff_longitude, 
                                                         dropoff_latitude)),
           transformEnvir = env, rowsPerRead=10000, reportProgress = 3)
used.time <- proc.time() - start.time
print(paste("It takes CPU Time=", round(used.time[1]+used.time[2],2), 
            " seconds, Elapsed Time=", round(used.time[3],2), " seconds to generate features using R functions.", sep=""))

### Feature engineering by calling a stored procedure in SQL Server

In [ ]:
# Do feature engineering through a SQL Query

ptypeColInfo <- list(
  payment_type = list(
    type = "factor",
    levels = c("CSH", "CRD", "DIS", "NOC", "UNK"),
    newLevels= c("CSH", "CRD", "DIS", "NOC", "UNK")
  )
)
# Alternatively, use a user defined function in SQL to create features
# Sometimes, feature engineering in SQL might be faster than R
# You need to choose the most efficient way based on real situation
# Here, featureEngineeringQuery is just a reference to the result from a SQL query. 
featureEngineeringQuery = "SELECT tipped, fare_amount, passenger_count,trip_time_in_secs,trip_distance, 
pickup_datetime, dropoff_datetime, 
dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance,
pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude, payment_type
FROM nyctaxi_sample
tablesample (70 percent) repeatable (98052)
"
featureDataSource <- RxSqlServerData(sqlQuery = featureEngineeringQuery, colInfo = ptypeColInfo,
                                    colClasses = c(pickup_longitude = "numeric", pickup_latitude = "numeric", 
                                                   dropoff_longitude = "numeric", dropoff_latitude = "numeric",
                                                   passenger_count  = "numeric", trip_distance  = "numeric",
                                                   trip_time_in_secs  = "numeric", direct_distance  = "numeric", fare_amount="numeric"),
                                    connectionString = connStr)

# summarize the feature table after the feature set is created
rxGetVarInfo(data = featureDataSource)

## Train logistic regression models to predict tipped or not (binary classification)

In [ ]:
################################
#        Training models       #
################################
# build classification model to predict tipped or not
system.time(logitObj <- rxLogit(tipped ~  passenger_count + trip_distance + trip_time_in_secs + direct_distance, data = featureDataSource))
summary(logitObj)

## Make predictions using the trained logistic regression model

In [ ]:
# predict and write the prediction results back to SQL Server table
outputtable <- paste0(username,"_scoreoutput")
scoredOutput <- RxSqlServerData(
  connectionString = connStr,
  table = outputtable
)

rxPredict(modelObject = logitObj, data = featureDataSource, outData = scoredOutput, 
          predVarNames = "Score", type = "response", writeModelVars = TRUE, overwrite = TRUE)

## Evaluate the model

In [ ]:
# plot ROC curve from local compute context
rxSetComputeContext('local') #change compute context to local, meaning that the plotting is done on local machine
scoreDF <- rxImport(scoredOutput) # Import data from SQL Server to a data frame on local machine. 
rxRocCurve("tipped", "Score", scoreDF) #plot the ROC curve
rxSetComputeContext(cc) # Switch the compute context to be in database

## Persist (operationalize) the model as a record in the database

In [ ]:
################################
#   Model operationalization   #
################################
# First, serialize a model and put it into a database table
modelbin <- serialize(logitObj, NULL)
modelbinstr <- paste(modelbin, collapse="")

library(RODBC)
conn <- odbcDriverConnect(connStr)

# Persist model by calling a stored procedure from SQL
q <- paste("EXEC PersistModel @m='", modelbinstr,"', @modeler='", user_name, "'", sep="")
sqlQuery(conn, q)

## Consume the model persisted in database through stored procedure execution

### Request-Response mode

In [ ]:
# Consume Model through Stored Proc Execution
# There are two stored Procs - One for predicting on single observation and another for predicting a batch of observations

# Single Observation prediction
q <- paste("EXEC PredictTipSingleMode @passenger_count=1, @trip_distance=2.5, @trip_time_in_secs=631, 
     @pickup_latitude=40.763958,@pickup_longitude=-73.973373, @dropoff_latitude=40.782139,@dropoff_longitude=-73.977303,
     @modeler='", user_name, "'", sep="")
sqlQuery(conn, q)

q <- paste("EXEC PredictTipSingleMode @passenger_count=9, @trip_distance=100, @trip_time_in_secs=5214, 
@pickup_latitude=40.75984, @pickup_longitude=-73.9754, @dropoff_latitude=41.0496, @dropoff_longitude=-73.54097,
@modeler='", user_name, "'", sep="")
sqlQuery(conn, q)

### Batch mode

In [ ]:
# Batch mode prediction. Score all data that were not part of the training dataset and select top 10 to preedict
input <- "N'select top 3 a.passenger_count as passenger_count, 
	a.trip_time_in_secs as trip_time_in_secs,
	a.trip_distance as trip_distance,
	a.dropoff_datetime as dropoff_datetime,  
	dbo.fnCalculateDistance(pickup_latitude, pickup_longitude, dropoff_latitude,dropoff_longitude) as direct_distance , fare_amount, payment_type
from
(
	select medallion, hack_license, pickup_datetime, passenger_count,trip_time_in_secs,trip_distance,  
		dropoff_datetime, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude, fare_amount, payment_type
	from nyctaxi_sample
)a
left outer join
(
select medallion, hack_license, pickup_datetime
from nyctaxi_sample
tablesample (70 percent) repeatable (98052)
)b
on a.medallion=b.medallion and a.hack_license=b.hack_license and a.pickup_datetime=b.pickup_datetime
where b.medallion is null
'"
q <- paste("EXEC PredictTipBatchMode @inquery = ", input, ", @modeler='", user_name, "'",sep="")
sqlQuery (conn, q)